In [7]:
import pandas as pd
import recordlinkage as rl
import time

import warnings
warnings.filterwarnings('ignore')

In [8]:
from recordlinkage.datasets import load_febrl4

set_a, set_b = load_febrl4()

In [19]:
random_a = set_a.sample(5)
random_b = set_b.sample(5)

In [22]:
random_a

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-918-org,jessica,michelmore,103,fimister circuit,swannie,mooroolbark,4174,nsw,19761101,6406097
rec-223-org,ky,kyriacou,30,ash place,kiloran,paddington,2665,nsw,19350528,5592621
rec-4825-org,lara,dunis,7,rodda place,dry rush,casino,4170,nsw,19580928,2469542
rec-1777-org,tommy,green,122,nanima road,port 44,lower templestowe,6064,vic,19901006,6395834
rec-919-org,liam,kayvas,61,biffin street,NaN,blacktown,2221,nsw,19970528,6098354


In [23]:
random_b

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-776-dup-0,maddiosn,morrison,18,derringtoncrescent,janefield,gunbwoer,6308,nsw,19021108,5581067
rec-1178-dup-0,rachel,dinnison,14,canopus crescent,NaN,barrack heights,3221,act,19410510,5292435
rec-2577-dup-0,noah,reetz,20,ellenborojgh street,lakes retirement estate,blacktown,3126,vic,19791110,2350308
rec-4044-dup-0,gus,holme,25,derry street,laure,allambie h eights,2024,qld,19580522,1510761
rec-4612-dup-0,alana,manson,52,NaN,vivigani,cloverdale,2118,vic,19460725,4207566


In [24]:
indexer = rl.Index()
indexer.block('state')   
pairs = indexer.index(random_a, random_b)
print(len(pairs))

6


In [26]:
compare = rl.Compare()  
compare.exact('state', 'state', label='state')
compare.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare.exact('soc_sec_id', 'soc_sec_id', label='soc_sec_id')   
compare.exact('postcode', 'postcode', label='postcode')
compare.string('given_name', 'given_name', threshold=0.75, 
                method='levenshtein', label='given_name')
compare.string('surname', 'surname', threshold=0.75, 
                method='levenshtein', label='surname')
compare.string('address_1', 'address_1', threshold=0.75,
                method='levenshtein', label='address')
matches = compare.compute(pairs, set_a, set_b)

In [27]:
matches

state  date_of_birth  soc_sec_id  postcode  \
rec_id_1     rec_id_2                                                     
rec-918-org  rec-776-dup-0       1              0           0         0   
rec-223-org  rec-776-dup-0       1              0           0         0   
rec-4825-org rec-776-dup-0       1              0           0         0   
rec-919-org  rec-776-dup-0       1              0           0         0   
rec-1777-org rec-2577-dup-0      1              0           0         0   
             rec-4612-dup-0      1              0           0         0   

                             given_name  surname  address  
rec_id_1     rec_id_2                                      
rec-918-org  rec-776-dup-0          0.0      0.0      0.0  
rec-223-org  rec-776-dup-0          0.0      0.0      0.0  
rec-4825-org rec-776-dup-0          0.0      0.0      0.0  
rec-919-org  rec-776-dup-0          0.0      0.0      0.0  
rec-1777-org rec-2577-dup-0         0.0      0.0      0.0  
             rec-4612-dup-0         0.0      0.0      0.0

In [37]:
matches.sum(axis=1).value_counts().sort_index(ascending=False)
matches

state  date_of_birth  soc_sec_id  postcode  \
rec_id_1     rec_id_2                                                     
rec-918-org  rec-776-dup-0       1              0           0         0   
rec-223-org  rec-776-dup-0       1              0           0         0   
rec-4825-org rec-776-dup-0       1              0           0         0   
rec-919-org  rec-776-dup-0       1              0           0         0   
rec-1777-org rec-2577-dup-0      1              0           0         0   
             rec-4612-dup-0      1              0           0         0   

                             given_name  surname  address  
rec_id_1     rec_id_2                                      
rec-918-org  rec-776-dup-0          0.0      0.0      0.0  
rec-223-org  rec-776-dup-0          0.0      0.0      0.0  
rec-4825-org rec-776-dup-0          0.0      0.0      0.0  
rec-919-org  rec-776-dup-0          0.0      0.0      0.0  
rec-1777-org rec-2577-dup-0         0.0      0.0      0.0  
             rec-4612-dup-0         0.0      0.0      0.0

In [40]:
  full_matches = matches[matches.sum(axis='columns') >= 1]

duplicates = full_matches.index.get_level_values('rec_id_2')
unique_b = set_b[~set_b.index.isin(duplicates)]
full_census = census_a.append(unique_b)

In [41]:
full_census

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
...,...,...,...,...,...,...,...,...,...,...
rec-4495-dup-0,connor,belperio,15,NaN,NaN,ryde,2570,nsw,19170518,5394641
rec-4211-dup-0,daniel,maspn,9,derrington crescent,el pedro caravan park,sunnybank,4350,vic,19500705,5525378
rec-3131-dup-0,samuel,crofs,613,banjine street,kurrajong vlge,pengzin,2230,qld,19410531,4467228
